# H2O

H2O genera un cluster con los nodos que nosotros le pongamos, con
la memoria que nosotros asignemos. 

Ese cluster trabaja todo y lo hace de manera más óptima, más rápida.

Es una manera de trabajar con muchos datos si tenemos problemas de memoria con un rendimiento un poco mejor 

Pero lo normal es que cuando vais a trabajar a un lado os habiliten o una gpu o un servidor en azure un algo donde vosotros podéis trabajar Lo tenéis que estar pasando las canutas con vuestro 16 gigas de ram

Librería programada en Java 100% 

Nos ayuda a optimizar todos los procesos que queramos hacer y
sobre todo los que son costosos de memoria

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score

In [2]:
import h2o
from h2o.estimators import H2ORandomForestEstimator

Lo levantamos en local y le decimos que utilice todos los recursos de la máquina para crearlo

Le dedicamos 4 gigas de memoria pra no quedarnos sin recursos

In [3]:
# Creación de un cluster local H2O
# ------------------------------------------------------------------------------
h2o.init(ip = "localhost",
         # -1 indica que se empleen todos los cores disponibles.
         nthreads = -1,
         # Máxima memoria disponible para el cluster.
         max_mem_size = "4g")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.301-b09, mixed mode)
  Starting server from C:\Python38\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\cx02202\AppData\Local\Temp\tmp7ff55zz8
  JVM stdout: C:\Users\cx02202\AppData\Local\Temp\tmp7ff55zz8\h2o_cx02202_started_from_python.out
  JVM stderr: C:\Users\cx02202\AppData\Local\Temp\tmp7ff55zz8\h2o_cx02202_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,09 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.4
H2O_cluster_version_age:,1 month and 12 days
H2O_cluster_name:,H2O_from_python_cx02202_zizfdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.556 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


#### Se eliminan los datos del cluster por si ya había sido iniciado.

Si ya lo has creado, y lo quieres reiniciar borrando todo lo que hay.


h2o.remove_all()

In [5]:
df_bank=pd.read_csv('../bank-full.csv', sep=';')

In [7]:
df_bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


Convertimos df_bank en un Dataframe de H2O

In [6]:
df_bank_h2o = h2o.H2OFrame(python_obj = df_bank, destination_frame = "df_bank_h2o")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [12]:
df_bank_h2o.head()

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no
43,technician,single,secondary,no,593,yes,no,unknown,5,may,55,1,-1,0,unknown,no


In [14]:
df_bank_h2o.shape

(45211, 17)

In [16]:
df_bank_h2o.col_names

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

In [18]:
df_bank_h2o.describe()

Rows:45211
Cols:17

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
type,int,enum,enum,enum,enum,int,enum,enum,enum,int,enum,int,int,int,int,enum,enum
mins,18.0,,,,,-8019.0,,,,1.0,,0.0,1.0,-1.0,0.0,,
mean,40.93621021432809,,,,,1362.272057685082,,,,15.806418791886935,,258.1630797814693,2.7638406582468984,40.1978279622216,0.5803233726305563,,
maxs,95.0,,,,,102127.0,,,,31.0,,4918.0,63.0,871.0,275.0,,
sigma,10.61876204097539,,,,,3044.7658291685234,,,,8.32247615304459,,257.5278122651711,3.0980208832791796,100.1287459905981,2.303441044931215,,
zeros,0,,,,,3514,,,,0,,3,0,0,36954,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,44.0,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,33.0,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no


In [19]:
df_bank_h2o['marital'].table()

marital,Count
divorced,5207
married,27214
single,12790


### Muestreo

In [32]:
df_bank_train_h2o, df_bank_test_h2o = df_bank_h2o.split_frame(ratios=[0.8],destination_frames= ["df_bank_train_h2o",
                                                        "df_bank_test_h2o"],
                                                        seed = 123)

In [29]:
df_bank_train_h2o.shape

(36227, 17)

In [37]:
df_bank_train_h2o.drop('y')

age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown
44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown
33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown
47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown
33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown
35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown
28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown
42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown
41,admin.,divorced,secondary,no,270,yes,no,unknown,5,may,222,1,-1,0,unknown
29,admin.,single,secondary,no,390,yes,no,unknown,5,may,137,1,-1,0,unknown


In [28]:
df_bank_test_h2o.shape

(8984, 17)

### Modelos

In [35]:
model_h2o = H2ORandomForestEstimator(ntrees=10,
                                    max_depth=5,
                                    min_rows=10,
                                    calibrate_model=True,
                                    calibration_frame=df_bank_test_h2o,
                                    binomial_double_trees=True)

In [45]:
df_bank_train_h2o['y'].asfactor()

y
no
no
no
no
no
no
no
no
no
no


In [46]:
df_bank_train_h2o.drop('y').col_names

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome']

In [47]:
model_h2o.train(x=df_bank_train_h2o.drop('y').col_names,
               y='y',
               training_frame=df_bank_train_h2o,
               validation_frame=df_bank_test_h2o)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1675340086299_1


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    10                 20                          8613                   5            5            5             26            32            29.65

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.07143322908577195
RMSE: 0.2672699554491151
LogLoss: 0.24150543139577066
Mean Per-Class Error: 0.19078149158071428
AUC: 0.8939062495083637
AUCPR: 0.5586624659180423
Gini: 0.7878124990167275

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19247724995076962
       no     yes    Error    Rate
-----  -----  -----  -------  ----------------
no     28354  3277   0.1036   (3277.0/31631.0)
yes    1173   3047   0.278    (1173.0/4220.0)
Total  29527  6324   0.1241   (4450.0/35851.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.192477     0.577959  231
max f2                       0.133023     0.675551  264
max f0point5                 0.403091     0.561123  137
max accuracy                 0.413872     0.900449  132
max precision                1            1         0
max recall                   0.00644424   1         397
max specificity              1            1         0
max absolute_mcc             0.18406      0.523068  235
max min_per_class_accuracy   0.11135      0.824645  278
max mean_per_class_accuracy  0.10683      0.825293  281
max tns                      1            31631     0
max fns                      1            4219      0
max fps                      0.00274657   31631     399
max tps                      0.00644424   4220      397
max tnr                      1            1         0
max fnr                      1            0.999763  0
max fpr                      0.00274657   1         399
max tpr                      0.00644424   1         397

Gains/Lift Table: Avg response rate: 11,77 %, avg score: 11,62 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100202                   0.678265           6.85766    6.85766            0.807163         0.739772   0.807163                    0.739772            0.0687148       0.0687148                  585.766   585.766            0.0665248
2        0.0200127                   0.58474            6.45415    6.65619            0.759669         0.626091   0.783448                    0.68301             0.0644934       0.133208                   545.415   565.619            0.128296
3        0.0300052                   0.529391           5.11638    6.14339            0.60221          0.553454   0.723091                    0.639864            0.0511257       0.184334                   411.638   514.339            0.174917
4        0.0400254                   0.496787           4.58738    5.75385            0.539945         0.512777   0.677241                    0.608049            0.0459662       0.2303                     358.738   475.385            0.215658
5        0.0500179                   0.470195           4.52964    5.50928            0.533149         0.483822   0.648455     